In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, AvgPool2D, Flatten, Dense
import PIL
from PIL import ImageOps
import itertools
import os
import shutil
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.image import imread, imsave
%matplotlib inline

# workstation used
ans = input('self or lab: ')
if ans == 'self':
    idi = '/home/neo11/Documents'
    print('using neo11')
else:
    idi = '/home/melcher/Documents/Nihal'
    print('using melcher')

2022-07-27 21:50:19.608271: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


self or lab: self
using neo11


In [ ]:
a = imread('train_cell.png')
img = np.array(a)
img = np.mean(img, -1)

U, S, VT = np.linalg.svd(img)
w = 2.8525 # beta = 1

s = S
sigma = np.median(s)
s.setflags(write = 1)

tau = sigma*w

for i in range(S.shape[0]):
    if s[i] < tau:
        s[i] = 0

n= 0
for j in s:
    if j == 0:
        print(n)
        break
    else:
        n = n+1

In [ ]:
a = imread('train_comp.png')
img = np.array(a)
img = np.mean(img, -1)

U, S, VT = np.linalg.svd(img)
w = 2.8525 # beta = 1

s = S
sigma = np.median(s)
s.setflags(write = 1)

tau = sigma*w

for i in range(S.shape[0]):
    if s[i] < tau:
        s[i] = 0
dataset_20722
n= 0
for j in s:
    if j == 0:
        print(n)
        break
    else:
        n = n+1

In [2]:
# since coffienct of the noise distribution is not know 
# using the 2nd method from IEEE 2014 SVD paper
class preprocessing():
    def __init__(self, input_di, output_di, ri):
        self.ri = ri
        self.d1 = input_di
        self.d2 = output_di
        self.arr = []
        self.w = 2.8525
        # we could add the genralized omega function but for our case beta = 1, and hence the value w = 2.8525 
        self.s_m = 0
        self.tau = 0
        self.r = 0
        self.i = 0
        self.original = []
        for j in os.listdir(self.d1):
            img = image.load_img(self.d1 + '/' + j)
            img = np.mean(np.array(img), -1)
            img = img/255
            U, S, VT = np.linalg.svd(img)
            s = S
            self.s_m = np.median(s)
            s.setflags(write =1)
            self.tau = self.s_m*self.w
            
            for i in range(S.shape[0]):
                if s[i] < self.tau:
                    s[i] = 0
            
            r = 0
            for j in s:
                if j == 0:
                    break
                else:
                    r = r + 1
            
            X = U[:,self.ri:r] @ np.diag(S[self.ri:r]) @ VT[self.ri:r,:]
            self.arr.append(X)
            self.original.append(img)
        
        self.arr = np.array(self.arr)
        self.original = np.array(self.original)
        print('pre-processing completed')
        print('initial parameter: ri = ' + str(ri))
    
    def original_arr(self):
        return self.original
    
    def converted_arr(self):
        return self.arr
    
    def noise_arr(self):
        noise = self.original - self.arr 
        return noise
    
    def save_img(self):
        for arr in self.arr:
            self.i = self.i + 1
            loc = os.path.join(self.d2 , "svd" +str(self.i) + '.png')
            mpimg.imsave(loc, arr, cmap ='gray')
        print('converted images saved in the second directory')
        print('please check the directory...')
    
    def print_converted_img(self):
        print('converted images')
        for arr in self.arr:
            plt.imshow(arr , cmap= 'gray')
            plt.show()
    
    def print_noise_img(self):
        print('noise as images')
        noise = self.original - self.arr
        for arr in noise:
            plt.imshow(arr , cmap= 'gray')
            plt.show()

    def random_dataset_generator(self, f, train_dir, test_dir):
        n = self.arr.shape[0]
        n1 = int(f*n)
        arr_list = [v for v in range(0,n)]
        rand1 = list(random.sample(range(0, n), n - n1))
        rand2 = list(set(arr_list) - set(rand1))
        i = 0 
        for i in range(len(rand1)):
            arr = self.arr[rand1[i]]
            i = i + 1
            loc = os.path.join(train_dir , "svd" +str(i) + '.png')
            mpimg.imsave(loc, arr, cmap ='gray')
        
        print('converted images saved in the train directory')
        
        i = 0
        for i in range(len(rand2)):
            arr = self.arr[rand2[i]]
            i = i + 1
            loc = os.path.join(test_dir , "svd" +str(i) + '.png')
            mpimg.imsave(loc, arr, cmap ='gray')
        
        print('converted images saved in the test directory')
        print('please check the directories...')

In [8]:
d1 = '/home/neo11/Documents/ML in microfluids/main dataset/compiled/Cell'
d2 = ''
d_train = '/home/neo11/Documents/ML in microfluids/main dataset/svd_compiled/train/Cell' 
d_test = '/home/neo11/Documents/ML in microfluids/main dataset/svd_compiled/test/cell'

a = preprocessing(d1, d2, 0)

a.random_dataset_generator(0.425, d_train, d_test) # for saving the images to the second directory

pre-processing completed
initial parameter: ri = 0
converted images saved in the train directory
converted images saved in the test directory
please check the directories...


In [7]:
di = idi + '/ML in microfluids/main dataset/cell_test'
do = idi + '/ML in microfluids/main dataset/cell_svd_test'

a = preprocessing(di, do, 0)

a.save_img()

pre-processing completed
initial parameter: ri = 0
converted images saved in the second directory
please check the directory...


In [ ]:
plt.imshow(a.converted_arr()[1], cmap = 'gray')

In [ ]:
plt.imshow(a.original_arr()[1], cmap = 'gray')

In [ ]:
plt.imshow(a.noise_arr()[1], cmap = 'gray')

In [ ]:
# ri =0
np.mean(abs(a.noise_arr()))

In [ ]:
# ri = 3
np.mean(abs(a.noise_arr()))

In [ ]:
img = imread('train_cell.png')
img = np.array(img)
img = np.mean(img, -1)
img = img/255

U, S, VT = np.linalg.svd(img)

r1 = 3
for r in range(r1+1,43):
    X = U[:,r1:r] @ np.diag(S[r1:r]) @ VT[r1:r,:]
    plt.imshow(X , cmap = 'gray')
    plt.title('rank = ' + str(r1) + ' to rank = ' + str(r))
    plt.show()

In [ ]:
img = imread('train_comp.png')
img = np.array(img)
img = np.mean(img, -1)
img = img/255

U, S, VT = np.linalg.svd(img)

r1 = 3
for r in range(r1+1,40):
    X = U[:,r1:r] @ np.diag(S[r1:r]) @ VT[r1:r,:]
    plt.imshow(X , cmap = 'gray')
    plt.title('rank = ' + str(r1) + ' to rank = ' + str(r))
    plt.show()

In [ ]:
img = imread('train_cell.png')
img = np.array(img)
img = np.mean(img, -1)
img = img/255

U, S, VT = np.linalg.svd(img)

r1 = 0
r = 30

X = U[:,r1:r] @ np.diag(S[r1:r]) @ VT[r1:r,:]
plt.imshow(255*X , cmap = 'gray')
plt.title('rank = ' + str(r))
plt.show()

In [ ]:
img = imread('train_comp.png')
img = np.array(img)
img = np.mean(img, -1)
img = img/255

U, S, VT = np.linalg.svd(img)

r1 = 0
r = 43

X = U[:,r1:r] @ np.diag(S[r1:r]) @ VT[r1:r,:]
plt.imshow(255*X , cmap = 'gray')
plt.title('rank = ' + str(r))
plt.show()

In [ ]:
import random

class testdata():
    def __init__(self, di_source, di_save):
        self.d1 = di_source
        self.d2 = di_save
        self.li = os.listdir(self.d1)
        
    def save(self, n):
        self.newli = []
        self.n = n
        self.l = random.sample(self.li, self.n)
        for j in self.l:
            img = image.load_img(self.d1 + '/' + j)
            img = np.mean(np.array(img), -1)
            img = img/255
            self.newli.append(img)
        self.newli = np.array(self.newli)
        self.i = 1
        for arr in self.newli:
            loc = os.path.join(self.d2 , "svd" + str(self.i) + '.png')
            mpimg.imsave(loc, arr, cmap ='gray')
            self.i = self.i + 1
        print('images saved at dir = ' + self.d2)
        
    def notused():
        li = list(self.newli)
        self.nli = [x for x in self.li if x not in self.newli]

In [ ]:
img_dir = idi + '/ML in microfluids/dataset/train/Cell'
save_dir = idi + '/ML in microfluids/'

testdata(img_dir, save_dir).save(69)